In [ ]:
!pip install transformers scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_file_path = '/content/drive/MyDrive/train.jsonl'

Mounted at /content/drive


In [ ]:
import json
import pandas as pd

def read_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Read the JSONL file into a DataFrame
df = read_jsonl(train_file_path)

# Display the DataFrame
print(df.head())

                                   uuid              postId  \
0  0af11f6b-c889-4520-9372-66ba25cb7657              532quh   
1  b1a1f63d-8853-4a11-89e8-6b2952a393ec  411701128456593408   
2  008b7b19-0445-4e16-8f9e-075b73f80ca4  380537005123190784   
3  31ecf93c-3e21-4c80-949b-aa549a046b93  844567852531286016   
4  31b108a3-c828-421a-a4b9-cf651e9ac859  814186311573766144   

                                            postText postPlatform  \
0  [Wes Welker Wanted Dinner With Tom Brady, But ...       reddit   
1   [NASA sets date for full recovery of ozone hole]      Twitter   
2  [This is what makes employees happy -- and it'...      Twitter   
3  [Passion is overrated — 7 work habits you need...      Twitter   
4  [The perfect way to cook rice so that it's per...      Twitter   

                                    targetParagraphs  \
0  [It’ll be just like old times this weekend for...   
1  [2070 is shaping up to be a great year for Mot...   
2  [Despite common belief, money isn't

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier

X = (df['postText'].apply(lambda x: ' '.join(x)) + ' ' + df['targetTitle']).values
#X = df['postText'].apply(lambda x: ' '.join(x)).values
y = df['tags']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Use TF-IDF for feature extraction
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_test_bin = mlb.transform(y_test)

# Train a logistic regression model for each tag
model = MultiOutputClassifier(LogisticRegression())
model.fit(X_train_tfidf, y_train_bin)

# Make predictions on the test set
y_pred_bin = model.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test_bin, y_pred_bin))
print("Classification Report:\n", classification_report(y_test_bin, y_pred_bin, target_names=mlb.classes_))

Accuracy: 0.3546875
Classification Report:
               precision    recall  f1-score   support

       multi       0.62      0.05      0.09       108
     passage       0.60      0.38      0.47       259
      phrase       0.61      0.45      0.52       273

   micro avg       0.61      0.35      0.45       640
   macro avg       0.61      0.29      0.36       640
weighted avg       0.61      0.35      0.42       640
 samples avg       0.35      0.35      0.35       640



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

X = (df['postText'].apply(lambda x: ' '.join(x)) + ' ' + df['targetTitle']).values
# X = df['postText'].apply(lambda x: ' '.join(x)).values
y = df['tags']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Use TF-IDF for feature extraction
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_test_bin = mlb.transform(y_test)

# Train a Random Forest classifier for each tag (replace LogisticRegression with RandomForestClassifier)
model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
model.fit(X_train_tfidf, y_train_bin)

# Make predictions on the test set
y_pred_bin = model.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test_bin, y_pred_bin))
print("Classification Report:\n", classification_report(y_test_bin, y_pred_bin, target_names=mlb.classes_))

Accuracy: 0.3734375
Classification Report:
               precision    recall  f1-score   support

       multi       0.64      0.17      0.26       108
     passage       0.56      0.42      0.48       259
      phrase       0.61      0.46      0.52       273

   micro avg       0.59      0.40      0.47       640
   macro avg       0.61      0.35      0.42       640
weighted avg       0.60      0.40      0.46       640
 samples avg       0.38      0.40      0.39       640



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC  # Import the Support Vector Machine classifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier

X = (df['postText'].apply(lambda x: ' '.join(x)) + ' ' + df['targetTitle']).values
y = df['tags']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use TF-IDF for feature extraction
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Import MultiLabelBinarizer and MultiOutputClassifier
mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_test_bin = mlb.transform(y_test)

# Train a Support Vector Machine classifier for each tag
model = MultiOutputClassifier(SVC())
model.fit(X_train_tfidf, y_train_bin)

# Make predictions on the test set
y_pred_bin = model.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test_bin, y_pred_bin))
print("Classification Report:\n", classification_report(y_test_bin, y_pred_bin, target_names=mlb.classes_))

Accuracy: 0.33125
Classification Report:
               precision    recall  f1-score   support

       multi       0.89      0.07      0.14       108
     passage       0.59      0.35      0.44       259
      phrase       0.63      0.42      0.51       273

   micro avg       0.62      0.33      0.43       640
   macro avg       0.70      0.28      0.36       640
weighted avg       0.66      0.33      0.42       640
 samples avg       0.33      0.33      0.33       640



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# # Convert predictions to binary format
# threshold = 0.5
# y_pred_bin_roberta = (np.array(all_preds) > threshold).astype(int)

# # Evaluate the model
# print("Accuracy:", accuracy_score(y_test_bin, y_pred_bin_roberta))
# print("Classification Report:\n", classification_report(y_test_bin, y_pred_bin_roberta, target_names=mlb.classes_))


Accuracy: 0.0
Classification Report:
               precision    recall  f1-score   support

       multi       0.17      1.00      0.29       108
     passage       0.40      1.00      0.58       259
      phrase       0.00      0.00      0.00       273

   micro avg       0.29      0.57      0.38       640
   macro avg       0.19      0.67      0.29       640
weighted avg       0.19      0.57      0.28       640
 samples avg       0.29      0.57      0.38       640



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# import json
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score
# from transformers import BertTokenizer, BertForSequenceClassification
# from torch.utils.data import DataLoader, TensorDataset

# # Load the training data
# with open('train.jsonl', 'r', encoding='utf-8') as file:
#     data = [json.loads(line) for line in file]

# # Separate features, labels, and spoiler types
# X = [' '.join(entry['targetParagraphs']) + ' ' + ' '.join(entry['postText']) for entry in data]
# y = [entry['tags'][0] for entry in data]  # Assuming each entry has a single tag

# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Load BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model_sequence_classification = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# # Tokenize and encode the training data
# X_train_encoded = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt')
# y_train_tensor = torch.tensor([{'phrase': 0, 'passage': 1, 'multi': 2}[tag] for tag in y_train])

# # Tokenize and encode the validation data
# X_val_encoded = tokenizer(X_val, padding=True, truncation=True, return_tensors='pt')
# y_val_tensor = torch.tensor([{'phrase': 0, 'passage': 1, 'multi': 2}[tag] for tag in y_val])

# # Create DataLoader for training and validation
# train_dataset = TensorDataset(X_train_encoded['input_ids'], X_train_encoded['attention_mask'], y_train_tensor)
# val_dataset = TensorDataset(X_val_encoded['input_ids'], X_val_encoded['attention_mask'], y_val_tensor)

# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# # Fine-tune the sequence classification model
# optimizer_sequence_classification = torch.optim.AdamW(model_sequence_classification.parameters(), lr=5e-5)

# for epoch in range(3):  # You may need more epochs depending on your dataset and model convergence
#     model_sequence_classification.train()
#     for batch in train_loader:
#         inputs, attention_mask, labels = batch
#         optimizer_sequence_classification.zero_grad()
#         outputs = model_sequence_classification(inputs, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer_sequence_classification.step()

# # Evaluate the sequence classification model on the validation set
# model_sequence_classification.eval()
# all_preds_sequence_classification = []

# with torch.no_grad():
#     for batch in val_loader:
#         inputs, attention_mask, labels = batch
#         outputs = model_sequence_classification(inputs, attention_mask=attention_mask)
#         logits = outputs.logits
#         preds = torch.argmax(logits, dim=1)
#         all_preds_sequence_classification.extend(preds.tolist())

# # Convert numerical labels back to spoiler types
# num_to_spoiler_type = {0: 'phrase', 1: 'passage', 2: 'multi'}
# final_predictions = [num_to_spoiler_type[pred] for pred in all_preds_sequence_classification]

# # Calculate and print accuracy and precision
# accuracy = accuracy_score(y_val, final_predictions)
# precision = precision_score(y_val, final_predictions, average='weighted')

# print(f'Accuracy: {accuracy:.4f}')
# print(f'Precision: {precision:.4f}')


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# import json
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score
# from transformers import BertTokenizer, BertForSequenceClassification
# from torch.utils.data import DataLoader, TensorDataset

# # Load the training data
# with open('/content/drive/MyDrive/train.jsonl', 'r', encoding='utf-8') as file:
#     data = [json.loads(line) for line in file]

# # Separate features and labels
# X = [' '.join(entry['targetParagraphs']) + ' ' + ' '.join(entry['postText']) for entry in data]
# y = [entry['tags'][0] for entry in data]  # Assuming each entry has a single tag

# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Load BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# # Tokenize and encode the training data
# X_train_encoded = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt')
# y_train_tensor = torch.tensor([int(tag == 'phrase') for tag in y_train])  # Convert tags to numerical labels

# # Tokenize and encode the validation data
# X_val_encoded = tokenizer(X_val, padding=True, truncation=True, return_tensors='pt')
# y_val_tensor = torch.tensor([int(tag == 'phrase') for tag in y_val])  # Convert tags to numerical labels

# # Create DataLoader for training and validation
# train_dataset = TensorDataset(X_train_encoded['input_ids'], X_train_encoded['attention_mask'], y_train_tensor)
# val_dataset = TensorDataset(X_val_encoded['input_ids'], X_val_encoded['attention_mask'], y_val_tensor)

# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# # Fine-tune the model
# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):  # You may need more epochs depending on your dataset and model convergence
#     model.train()
#     for batch in train_loader:
#         inputs, attention_mask, labels = batch
#         optimizer.zero_grad()
#         outputs = model(inputs, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

# # Evaluate the model on the validation set
# model.eval()
# all_preds = []

# with torch.no_grad():
#     for batch in val_loader:
#         inputs, attention_mask, labels = batch
#         outputs = model(inputs, attention_mask=attention_mask)
#         logits = outputs.logits
#         preds = torch.argmax(logits, dim=1)
#         all_preds.extend(preds.tolist())

# # Calculate and print accuracy and precision
# accuracy = accuracy_score(y_val_tensor.tolist(), all_preds)
# precision = precision_score(y_val_tensor.tolist(), all_preds, average='weighted')

# print(f'Accuracy: {accuracy:.4f}')
# print(f'Precision: {precision:.4f}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
